## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-14-14-53-40 +0000,bbc,World's oldest president to seek eighth term,https://www.bbc.com/news/articles/ckglpnk0kqko
1,2025-07-14-14-50-24 +0000,bbc,"Shoddy that Trump can't address MPs, says Farage",https://www.bbc.com/news/articles/cx2l1gjw49yo
2,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...
3,2025-07-14-14-41-07 +0000,bbc,Body of former Nigerian leader Buhari to be fl...,https://www.bbc.com/news/articles/c93kxl08glwo
4,2025-07-14-14-31-32 +0000,bbc,Two women killed at Kentucky church as gunman ...,https://www.bbc.com/news/articles/cm2l5jn6254o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2309/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,12
14,trade,7
52,ukraine,5
38,killed,4
139,tariffs,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
2,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...,48
39,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...,44
53,2025-07-13-16-57-56 +0000,nyt,How Trump’s Latest Tariff Threat Risks Deeper ...,https://www.nytimes.com/2025/07/13/business/tr...,28
7,2025-07-14-14-18-31 +0000,nyt,Trump Reluctantly Comes Around to Backing Ukra...,https://www.nytimes.com/2025/07/14/us/politics...,27
20,2025-07-14-11-57-34 +0000,nyt,Trump’s E.U. Tariffs Would Hobble Trade With U...,https://www.nytimes.com/2025/07/14/world/europ...,27


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
2,48,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...
56,19,2025-07-13-16-29-05 +0000,bbc,Gaza officials say children killed in strike a...,https://www.bbc.com/news/articles/c0rvxjnvv71o
39,18,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...
30,18,2025-07-14-09-30-00 +0000,wsj,Food banks across the country were already str...,https://www.wsj.com/economy/food-banks-are-run...
33,17,2025-07-14-06-08-00 +0000,wsj,"China’s exports grew at a faster clip in June,...",https://www.wsj.com/economy/trade/chinas-expor...
5,13,2025-07-14-14-23-56 +0000,nyt,Zelensky Announces Plan for Major Government S...,https://www.nytimes.com/2025/07/14/world/europ...
54,12,2025-07-13-16-51-25 +0000,bbc,Buhari - the austere Nigerian military ruler w...,https://www.bbc.com/news/articles/c89781qxnzdo
29,12,2025-07-14-09-42-00 +0000,wsj,Higher Tariffs Would Likely Spur ECB to Furthe...,https://www.wsj.com/articles/higher-tariffs-wo...
59,12,2025-07-13-15-20-08 +0000,bbc,Texas camp buildings were removed from map sho...,https://www.bbc.com/news/articles/ce8zjk5yx8wo
4,11,2025-07-14-14-31-32 +0000,bbc,Two women killed at Kentucky church as gunman ...,https://www.bbc.com/news/articles/cm2l5jn6254o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
